In [13]:
import pandas as pd
import numpy as np
from pandas_datareader import data
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import math
import six
import sys
sys.modules['sklearn.externals.six'] = six
import statistics
from fitness import Fitness
import mlrose
import yfinance as yf
yf.pdr_override()

dataset = pd.read_csv('excel/acoes.csv')
dataset.drop(columns=["BOVA11.SA"], inplace=True)
dataset


,Date,VALE3.SA,KEPL3.SA,VULC3.SA,LEVE3.SA,GGBR4.SA,LREN3.SA,ALSO3.SA,DIRR3.SA,MILS3.SA,...,CLSC4.SA,CMIG4.SA,ITSA4.SA,BBAS3.SA,TASA4.SA,SLCE3.SA,B3SA3.SA,PETR4.SA,SUZB3.SA,KLBN4.SA
0,2012-03-01,22.652565,1.244356,6.405912,9.398795,11.194284,8.337506,22.175179,5.758006,20.450829,...,21.851042,4.780592,3.164979,13.042416,18.954088,3.407774,2.430807,7.992807,17.512335,1.052998
1,2012-03-02,22.858259,1.244356,6.405912,9.354464,11.317574,8.223716,22.547600,6.144378,20.263292,...,21.661739,4.754763,3.209084,13.288682,18.863834,3.409905,2.499368,8.127735,17.512335,1.110546
2,2012-03-05,22.189741,1.196496,6.578115,9.230327,11.120316,8.182797,22.280426,6.042418,20.147192,...,21.580612,4.849851,3.175359,13.311914,18.863834,3.450398,2.493135,7.899643,17.512335,1.090955
3,2012-03-06,21.114967,1.292216,6.405912,9.381063,10.577860,8.176403,22.183271,6.037053,20.066813,...,21.120872,4.691759,3.084561,13.079591,17.871000,3.495153,2.430807,7.629788,17.512335,1.057896
4,2012-03-07,21.084114,1.531515,6.578115,9.404115,10.540873,8.355404,22.385677,6.063884,20.129332,...,21.120872,4.659293,3.133849,13.265442,18.322290,3.495153,2.503522,7.703676,17.512335,1.100750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2893,2023-10-24,63.990002,10.750000,18.150000,40.799999,21.549999,12.380000,23.330000,17.230000,10.800000,...,62.889999,11.930000,8.780000,49.040001,15.620000,36.849998,11.600000,35.880001,52.509998,4.391650
2894,2023-10-25,63.930000,10.390000,18.160000,40.459999,21.469999,12.350000,23.330000,17.000000,10.500000,...,62.720001,11.850000,8.710000,49.049999,15.380000,36.919998,11.540000,36.070000,52.599998,4.371912
2895,2023-10-26,65.300003,10.670000,18.280001,40.299999,21.809999,12.680000,23.330000,17.719999,10.710000,...,62.720001,12.160000,8.900000,50.070000,15.380000,37.290001,11.730000,35.700001,52.709999,4.401519
2896,2023-10-27,67.570000,10.500000,17.500000,40.189999,21.840000,12.210000,23.129999,17.799999,10.800000,...,63.880001,11.930000,8.730000,48.799999,14.890000,36.529999,11.200000,35.439999,51.919998,4.352175


In [14]:
dinheiro_total = 5000
taxas_selic = [12.5, 9.25, 7.25, 7.75, 9.5, 14.25, 14.25, 14.25, 6.5, 5.25, 2.25, 7.75, 11.75, 12.5]

valores = [dinheiro_total]

for taxa in taxas_selic:
    novo_valor = valores[-1] + (valores[-1] * taxa / 100)
    valores.append(novo_valor)

valor_2023 = valores[-1]
taxa_selic_historico = np.array(taxas_selic)
dataset_original = dataset.copy()
sem_risco = taxa_selic_historico.mean() / 100
rf = sem_risco

pesos = np.array([0.04579877, 0.10065911, 0.05203524, 0.07009186, 0.09986174,
         0.05894185, 0.01878224, 0.02098791, 0.01420743, 0.01101616,
         0.01289593, 0.06706893, 0.09493909, 0.01703658,
        0.02083494, 0.02857037, 0.0379588 , 0.01738156, 0.06394153,
       0.01380989000000001 , 0.13318007])  
# (dataset['taxa de retorno'].mean() - taxa_selic_historico.mean()/100)/ dataset['taxa de retorno'].std() * math.sqrt(246) #sharp ratio validando historico da selic


In [15]:
len(dataset.columns[1:])
dataset.columns
dataset_original

,Date,VALE3.SA,KEPL3.SA,VULC3.SA,LEVE3.SA,GGBR4.SA,LREN3.SA,ALSO3.SA,DIRR3.SA,MILS3.SA,...,CLSC4.SA,CMIG4.SA,ITSA4.SA,BBAS3.SA,TASA4.SA,SLCE3.SA,B3SA3.SA,PETR4.SA,SUZB3.SA,KLBN4.SA
0,2012-03-01,22.652565,1.244356,6.405912,9.398795,11.194284,8.337506,22.175179,5.758006,20.450829,...,21.851042,4.780592,3.164979,13.042416,18.954088,3.407774,2.430807,7.992807,17.512335,1.052998
1,2012-03-02,22.858259,1.244356,6.405912,9.354464,11.317574,8.223716,22.547600,6.144378,20.263292,...,21.661739,4.754763,3.209084,13.288682,18.863834,3.409905,2.499368,8.127735,17.512335,1.110546
2,2012-03-05,22.189741,1.196496,6.578115,9.230327,11.120316,8.182797,22.280426,6.042418,20.147192,...,21.580612,4.849851,3.175359,13.311914,18.863834,3.450398,2.493135,7.899643,17.512335,1.090955
3,2012-03-06,21.114967,1.292216,6.405912,9.381063,10.577860,8.176403,22.183271,6.037053,20.066813,...,21.120872,4.691759,3.084561,13.079591,17.871000,3.495153,2.430807,7.629788,17.512335,1.057896
4,2012-03-07,21.084114,1.531515,6.578115,9.404115,10.540873,8.355404,22.385677,6.063884,20.129332,...,21.120872,4.659293,3.133849,13.265442,18.322290,3.495153,2.503522,7.703676,17.512335,1.100750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2893,2023-10-24,63.990002,10.750000,18.150000,40.799999,21.549999,12.380000,23.330000,17.230000,10.800000,...,62.889999,11.930000,8.780000,49.040001,15.620000,36.849998,11.600000,35.880001,52.509998,4.391650
2894,2023-10-25,63.930000,10.390000,18.160000,40.459999,21.469999,12.350000,23.330000,17.000000,10.500000,...,62.720001,11.850000,8.710000,49.049999,15.380000,36.919998,11.540000,36.070000,52.599998,4.371912
2895,2023-10-26,65.300003,10.670000,18.280001,40.299999,21.809999,12.680000,23.330000,17.719999,10.710000,...,62.720001,12.160000,8.900000,50.070000,15.380000,37.290001,11.730000,35.700001,52.709999,4.401519
2896,2023-10-27,67.570000,10.500000,17.500000,40.189999,21.840000,12.210000,23.129999,17.799999,10.800000,...,63.880001,11.930000,8.730000,48.799999,14.890000,36.529999,11.200000,35.439999,51.919998,4.352175


In [16]:
# def fitness_function(solucao):
#     a = []
    

#     for j in range(12,24):
        
#         dataset = pd.read_csv(f'excel/acoes20{j}.csv')
#         pesos = solucao / solucao.sum()
#         dataset.drop(columns=["BOVA11.SA"], inplace=True)

#         for i in dataset.columns[1:]:
#             dataset[i] = (dataset[i] / dataset[i][0]) # normalizador das colunas

#         for i, acao in enumerate(dataset.columns[1:]):
#             dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

#         dataset.drop(labels = ['Date'], axis = 1, inplace=True)
#         dataset['soma valor'] = dataset.sum(axis = 1)
        
        
#         dataset['taxa retorno'] = 0.0
#         dataset["taxa retorno"] = dataset["soma valor"].pct_change()
#         risk_free = sem_risco/252
#         dataset["retorno Calc"] = dataset["taxa retorno"] - risk_free
#         sharpe_ratio = np.sqrt(252) * (dataset["retorno Calc"].mean()/dataset["retorno Calc"].std())  
#         a.append(sharpe_ratio)
        
#     peso_valor = [1,1,1,1,1,1,2,2,3,3,4,5]
#     np.multiply(a, peso_valor)
#     sharpe_ratio = statistics.mean(a)/25   
#     return sharpe_ratio


# def fitness_function(solucao):
  
#   dataset = dataset_original.copy()
#   pesos = solucao / solucao.sum()

#   for i in dataset.columns[1:]:
#     dataset[i] = (dataset[i] / dataset[i][0]) # normalizador das colunas

#   for i, acao in enumerate(dataset.columns[1:]):
#     dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

#   dataset.drop(labels = ['Date'], axis = 1, inplace=True)
#   dataset['soma valor'] = dataset.sum(axis = 1)
  
  
#   dataset['taxa retorno'] = 0.0
#   dataset["taxa retorno"] = dataset["soma valor"].pct_change()
#   risk_free = sem_risco/252
#   dataset["retorno Calc"] = dataset["taxa retorno"] - risk_free
#   sharpe_ratio = np.sqrt(252) * (dataset["retorno Calc"].mean()/dataset["retorno Calc"].std())  

#   return sharpe_ratio


In [17]:
def alocacao_ativos(dataset, dinheiro_total, seed = 0, melhores_pesos = []):
    if seed != 0:
        np.random.seed(seed)

    dataset = dataset.copy()

    if len(melhores_pesos) > 0:
        pesos = melhores_pesos
    else:  
        pesos = np.random.random(len(dataset.columns) - 1)
    #print(pesos, pesos.sum())
        pesos = pesos / pesos.sum()
    #print(pesos, pesos.sum())
    
 
    colunas = dataset.columns[1:]

    for i in colunas:
        dataset[i] = dataset[i] / dataset[i][0]

    for i, acao in enumerate(dataset.columns[1:]):
        dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

    dataset['SOMA VALOR'] = dataset[colunas].sum(axis=1)

    datas = dataset['Date']

    dataset.drop(columns=['Date'], inplace=True)
    dataset['taxa de retorno'] = 0.0

    for i in range(1, len(dataset)):
        dataset['taxa de retorno'][i] = ((dataset['SOMA VALOR'][i] / dataset['SOMA VALOR'][i-1]) - 1 )*100

    acoes_pesos = pd.DataFrame(data= {'Ações': colunas, 'Pesos': pesos*100})

    return dataset, datas, acoes_pesos , dataset.loc[len(dataset)-1]['SOMA VALOR']

In [72]:
#np.random.seed(3)
pesos = np.random.random(len(dataset_original.columns) - 1)
pesos = pesos / pesos.sum()
pesos

array([0.06398168, 0.0172765 , 0.08965997, 0.05411586, 0.00223983,
       0.08607066, 0.07223122, 0.06896497, 0.00543013, 0.07400468,
       0.03545926, 0.08544232, 0.01637094, 0.03518856, 0.01081627,
       0.00192564, 0.03394362, 0.07336818, 0.04403681, 0.03125563,
       0.09821727])

In [35]:
def visualiza_alocacao(solucao):
  colunas = dataset_original.columns[1:]
  for i in range(len(solucao)):
    print(colunas[i], solucao[i] * 100)
visualiza_alocacao(pesos)    

VALE3.SA 6.190690525621148
KEPL3.SA 7.959224237655088
VULC3.SA 3.2696224924400914
LEVE3.SA 5.741444549699863
GGBR4.SA 10.036273239827425
LREN3.SA 10.073882104314446
ALSO3.SA 1.4115155268595037
DIRR3.SA 2.329305388048063
MILS3.SA 0.5784653978625593
RANI3.SA 4.954480236640066
CSUD3.SA 0.3357935365455789
CLSC4.SA 5.134574951774411
CMIG4.SA 7.296051576354643
ITSA4.SA 3.1300565491194368
BBAS3.SA 7.600763900923581
TASA4.SA 6.64099995410924
SLCE3.SA 0.2695442581115802
B3SA3.SA 6.281237985036397
PETR4.SA 2.9138667001311664
SUZB3.SA 4.66552801942869
KLBN4.SA 3.186678869496995


In [20]:
fitness_obj = Fitness()
fitness = mlrose.CustomFitness(fitness_obj.fitness_function)

In [21]:
problema_maximizacao = mlrose.ContinuousOpt(length=21, fitness_fn=fitness,
                                            maximize = True, min_val = 0, max_val = 1)

In [22]:
problema_minimizacao = mlrose.ContinuousOpt(length=21, fitness_fn=fitness,
                                            maximize = False, min_val = 0, max_val = 1)

In [23]:
melhor_solucao, melhor_custo = mlrose.hill_climb(problema_maximizacao, random_state = 1) ## hill climb
melhor_solucao, melhor_custo

KeyboardInterrupt: 

In [ ]:
melhor_solucao = melhor_solucao / melhor_solucao.sum()
melhor_solucao, melhor_solucao.sum()

(array([0.        , 0.29981064, 0.06666269, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.11286762,
        0.07301872, 0.        , 0.        , 0.        , 0.00912344,
        0.        , 0.23895068, 0.05286322, 0.14670299, 0.        ,
        0.        ]),
 0.9999999999999998)

In [ ]:
visualiza_alocacao(melhor_solucao)

VALE3.SA 0.0
KEPL3.SA 29.98106410152368
VULC3.SA 6.666268776705726
LEVE3.SA 0.0
GGBR4.SA 0.0
LREN3.SA 0.0
ALSO3.SA 0.0
DIRR3.SA 0.0
MILS3.SA 0.0
RANI3.SA 11.286762467581715
CSUD3.SA 7.301871985586153
CLSC4.SA 0.0
CMIG4.SA 0.0
ITSA4.SA 0.0
BBAS3.SA 0.9123435413927535
TASA4.SA 0.0
SLCE3.SA 23.895068066777338
B3SA3.SA 5.286322132102087
PETR4.SA 14.67029892833053
SUZB3.SA 0.0
KLBN4.SA 0.0


In [ ]:
_, _, _, soma_valor = alocacao_ativos(dataset, 5000, melhores_pesos=melhor_solucao)
soma_valor

43259.06090317872

In [ ]:
## Simulated annealing
melhor_solucao, melhor_custo = mlrose.simulated_annealing(problema_maximizacao, random_state = 1) 
melhor_solucao = melhor_solucao / melhor_solucao.sum()
melhor_solucao, melhor_custo

KeyboardInterrupt: 

In [ ]:
visualiza_alocacao(melhor_solucao)

In [ ]:
_, _, _, soma_valor = alocacao_ativos(dataset, 5000, melhores_pesos=melhor_solucao)
soma_valor

In [ ]:
#algoritmo genético 
problema_maximizacao_ag = mlrose.ContinuousOpt(length = 21, fitness_fn = fitness, 
                                               maximize = True, min_val = 0.05, max_val = 0.2)
melhor_solucao, melhor_custo = mlrose.genetic_alg(problema_maximizacao_ag, random_state = 1, pop_size=2000, mutation_prob= 0.1, max_attempts = 20)
melhor_solucao = melhor_solucao / melhor_solucao.sum()
melhor_solucao, melhor_custo


(array([0.04218554, 0.07669078, 0.05286431, 0.08773327, 0.02379668,
        0.02958202, 0.02421602, 0.03531603, 0.04751383, 0.08651604,
        0.02517897, 0.03004437, 0.04355555, 0.04228403, 0.04938722,
        0.02524045, 0.09086534, 0.08808711, 0.03708232, 0.02364835,
        0.03821178]),
 0.0214684004298996)

In [ ]:
visualiza_alocacao(melhor_solucao)

VALE3.SA 4.218553966915522
KEPL3.SA 7.669077573511901
VULC3.SA 5.286431151631675
LEVE3.SA 8.773326785880272
GGBR4.SA 2.37966762188274
LREN3.SA 2.958201519486613
ALSO3.SA 2.421602026636973
DIRR3.SA 3.5316032548411496
MILS3.SA 4.751383204915487
RANI3.SA 8.651603958504888
CSUD3.SA 2.5178966798226248
CLSC4.SA 3.004437340544609
CMIG4.SA 4.355555139745632
ITSA4.SA 4.228402844028734
BBAS3.SA 4.938721642376807
TASA4.SA 2.5240453940159835
SLCE3.SA 9.086534109349264
B3SA3.SA 8.808711290333529
PETR4.SA 3.7082316010754623
SUZB3.SA 2.364835070820206
KLBN4.SA 3.821177823679937


In [ ]:
_, _, _, soma_valor = alocacao_ativos(dataset, 5000, melhores_pesos=melhor_solucao)
soma_valor

28647.674429584436